## This script makes intersected .bed files for replicates as "confident editing clusters"

In [1]:
import os
cwd = os.getcwd()
main_dir = cwd
print(main_dir)

/tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks


In [2]:
import pybedtools
import os
from glob import glob

peakcaller_bed_dir = main_dir + '/flare_filtered/peak_calling/'
all_sample=sorted(glob(os.path.join(peakcaller_bed_dir, '*/*merged_sorted_peaks.fdr_0.1.d_15.merged.bed')))

sample_ids = [i.split('/')[-1].split('_merge')[0] for i in all_sample]

In [3]:
import pandas as pd

sample_id_to_label = pd.read_csv(main_dir + '/sample_id_to_label-pub.txt', sep = '\t', header = None, index_col = 0).to_dict().get(1)
unique_sample_lst = []

for sample in sample_id_to_label: 
    unique_sample =  sample_id_to_label.get(sample).split(' rep')[0]
    unique_sample_lst.append(unique_sample) if unique_sample not in unique_sample_lst else unique_sample_lst

print(unique_sample_lst)
print(sample_id_to_label)

['RBFOX2 INSCRIBE PFA', 'TDP43 INSCRIBE PFA', 'Enzyme Only PFA', 'Buffer Only PFA']
{'isSTAMP10_01': 'RBFOX2 INSCRIBE PFA rep1', 'isSTAMP10_02': 'RBFOX2 INSCRIBE PFA rep2', 'isSTAMP10_03': 'RBFOX2 INSCRIBE PFA rep3', 'isSTAMP10_07': 'TDP43 INSCRIBE PFA rep1', 'isSTAMP10_08': 'TDP43 INSCRIBE PFA rep2', 'isSTAMP10_09': 'TDP43 INSCRIBE PFA rep3', 'isSTAMP10_13': 'Enzyme Only PFA rep1', 'isSTAMP10_14': 'Enzyme Only PFA rep2', 'isSTAMP10_15': 'Enzyme Only PFA rep3', 'isSTAMP10_16': 'Buffer Only PFA rep1', 'isSTAMP10_17': 'Buffer Only PFA rep2', 'isSTAMP10_18': 'Buffer Only PFA rep3'}


In [5]:
# For 3 replicates
import subprocess

confident_peaks_dir = main_dir + '/confident_clusters/'
subprocess.run(["mkdir", confident_peaks_dir])

for unique_sample in unique_sample_lst:
    if 'PFA' in unique_sample:

        for bed in all_sample:
            bed_id = bed.split('/')[-1].split('_merge')[0]
            # print(bed_id)
            # print(sample_id_to_label.get(bed_id))
            if unique_sample + ' rep1' in sample_id_to_label.get(bed_id):
                bed1 = bed
            if unique_sample + ' rep2' in sample_id_to_label.get(bed_id):
                bed2 = bed
            if unique_sample + ' rep3' in sample_id_to_label.get(bed_id):
                bed3 = bed

        bed_file_name = confident_peaks_dir + '/' + unique_sample.replace(' ', '_') + '_confident_clusters.bed'
        print('Sample name: ' + unique_sample)

        rep1 = pybedtools.example_bedtool(bed1)
        rep2 = pybedtools.example_bedtool(bed2)
        rep3 = pybedtools.example_bedtool(bed3)

        print('  Cluster counts in 3 replicates:')
        print('\t', len(rep1), len(rep2), len(rep3))

        rep1_rep2 = rep1.intersect(rep2, wa=True, u=True, s=True)
        rep1_rep2_rep3 = rep3.intersect(rep1_rep2, wa=True, u=True, s=True, output = bed_file_name)

        print('\t  Cluster counts after rep1 x rep2:')
        print('\t  ', len(rep1_rep2))
        print('\t\t  Cluster counts after rep1 x rep2 x rep3:')
        print('\t\t  ', len(rep1_rep2_rep3))




Sample name: RBFOX2 INSCRIBE PFA
  Cluster counts in 3 replicates:
	 144721 146234 183136
	  Cluster counts after rep1 x rep2:
	   14123
		  Cluster counts after rep1 x rep2 x rep3:
		   3727
Sample name: TDP43 INSCRIBE PFA
  Cluster counts in 3 replicates:
	 151547 127868 163817
	  Cluster counts after rep1 x rep2:
	   13084
		  Cluster counts after rep1 x rep2 x rep3:
		   3301
Sample name: Enzyme Only PFA
  Cluster counts in 3 replicates:
	 119019 130855 96054
	  Cluster counts after rep1 x rep2:
	   11340
		  Cluster counts after rep1 x rep2 x rep3:
		   2431
Sample name: Buffer Only PFA
  Cluster counts in 3 replicates:
	 123733 106364 119226
	  Cluster counts after rep1 x rep2:
	   9989
		  Cluster counts after rep1 x rep2 x rep3:
		   2491
